In [0]:
# Create functions for normalize table and unpivot
def normalize_fecha(df):
    return (df
        .withColumn("_ym", to_date(concat_ws("-", col("FECHA"), lit("01")), "yyyy-MM-dd"))
        .withColumn("FECHA", last_day(col("_ym")))
        .drop("_ym")
    )
def unpivot_to_long(df_normalize):
    cols = [c for c in df_normalize.columns if c != "FECHA"]
    N = len(cols)
    pairs = ", ".join([f"'{c}', `{c}`" for c in cols])
    unpivot_df = df_normalize\
        .select(
            col("FECHA"),
            expr(f"stack({N}, {pairs}) AS (POZO, PRODUCION_MBD)")
        )

    return unpivot_df

def add_flags(df_unpivot):
    withColumn("PRODUCION_MBD", col("PRODUCION_MBD").cast("double"))\
    .withColumn("FLAG_NULL", when(col("PRODUCION_MBD")).IsNull(),1).otherwise(0)\
    .withColumn("FLAG_ZERO", when(col("PRODUCION_MBD") == 0),1).otherwise(0)
    return df_unpivot

In [0]:
def process_silver(microDF, batchId):
    if microDF.rdd.isEmpty():
        return

    # Normalize and unpivot table    
    df_normalize = normalize_fecha(microDF)
    df_unpivot = unpivot_to_long(df_normalize)
    df = add_flags(df_unpivot)

    #Collect min and max date in order to filter historic table
    minmax = df.agg(min("FECHA").alias("minf"),max("FECHA").alias("maxf")).collect()[0]
    minf, maxf = minmax["minf"], minmax["maxf"]

    hist = spark.table(silver_table).filter(
        (col("FECHA") >= add_months(lit(minf), -12)) & (col("FECHA") <= add_months(lit(maxf), 0))
    ).select("POZO", "FECHA", "PRODUCION_MBD")

    #unionByName hist + new data 
    union_for_stats = df.select("POZO", "PRODUCCION_MBD").unionByName(
        hist.select("POZO","PRODUCCION_MBD")
    )

    #Calculate IQR in order to add flags 
    stats = (
        union_for_stats
        .filter(col("PRODUCCION_MBD").isNotNull())
        .groupBy("POZO")
        .agg(
            expr("percentile_approx(PRODUCCION_MBD, 0.25, 1000)").alias("Q1"),
            expr("percentile_approx(PRODUCCION_MBD, 0.75, 1000)").alias("Q3")
        )
        .withColumn("IQR", col("Q3") - col("Q1"))
        .withColumn("LOW_B", col("Q1") - 1.5*col("IQR"))
        .withColumn("HIGH_B", col("Q3") + 1.5*col("IQR"))
    )

    # add flags when the new data is less than lowb or mayor than highb
    df2 = (
        df.join(stats, on = "POZO", how = "left")
        .withColumn("FLAG_OUTLIER_IQR",
            when(
                (col("PRODUCCION_MBD").isNotNull() &
               ((col("PRODUCCION_MBD") < col("LOW_B")) | (col("PRODUCCION_MBD") > col("HIGH_B"))),
                1)).otherwise(0)
            )
        .drop("Q1","Q3","IQR","LOW_B","HIGH_B")
    )

    #Geting historic table 
    prev = (spark.table(silver_table)
    .select("POZO","FECHA","PRODUCCION_MBD")
    .withColumnRenamed("FECHA","FECHA_PREV")
    .withColumnRenamed("PRODUCCION_MBD","PROD_PREV")
    )

    #Calculate PCT CHANGE and add flag outiler Mom
    df3 = (df2
      .withColumn("FECHA_PREV", add_months("FECHA", -1))
      .join(prev, on=["POZO","FECHA_PREV"], how="left")
      .withColumn("PCT_CHANGE",
        when(col("PROD_PREV").isNull(), None)
        .otherwise(abs(col("PRODUCCION_MBD") - col("PROD_PREV")) /
        .when(col("PROD_PREV") == 0, None).otherwise(col("PROD_PREV")))
      )
      .withColumn("FLAG_OUTLIER_MOM",
        when(col("PCT_CHANGE") > lit(2.0), 1).otherwise(0))
    )

    #Get historic table + new data 
    hist_for_streak = spark.table(silver_table).select("POZO","FECHA","PRODUCCION_MBD")
    union_for_streak = (hist_for_streak.unionByName(df3.select("POZO","FECHA","PRODUCCION_MBD"))
        .dropDuplicates(["POZO","FECHA"])
    )

    #Add FLAG INACTIVE when pozos tengan rachas de ceros 
    w = Window.partitionBy("POZO").orderBy("FECHA")
    tmp = (
        union_for_streak
            .withColumn("IS_ZERO", when(col("PRODUCCION_MBD")==0,1).otherwise(0))
            .withColumn("grp", sum(when(col("IS_ZERO") == 0, 1).otherwise(0)).over(w))
    )


    wgrp = Window.partitionBy("POZO","grp")
    streaked = (
        tmp.withColumn("STREAK_ZERO", sum("IS_ZERO").over(wgrp))
        .select("POZO","FECHA","STREAK_ZERO")
    )

    df4 = (
        df3.join(streaked, on = ["POZO","FECHA"], how="left")
        .withColum("FLAG_INACTIVO", when(col("STREAK_ZERO" >= lit(6),1).otherwise(0)))
        .drop("STREAK_ZERO","FECHA_PREV","PROD_PREV","PCT_CHANGE")
    )

    #Select columns and add año, mes, ingestion
    out = (
        df4.withColumn("ANIO", year("FECHA"))
        .withColumn("MES", month("FECHA"))
        .withColumn("INGESTION_TS", current_timestamp())
        .select("FECHA","ANIO","MES","POZO","PRODUCCION_MBD","FLAG_NULL","FLAG_ZERO",
                "FLAG_OUTLIER_IQR","FLAG_OUTLIER_MOM","FLAG_INACTIVO",
                "DATA_SOURCE","INGESTION_TS")
    )
     
    out.createOrReplaceTempView("silver_upserts")

    #SCD TYPE 1 
    spark.sql(f"""
    MERGE INTO {silver_table} AS t
    USING silver_upserts as s
    ON t.FECHA = s.FECHA AND t.POZO = s.POZO
    WHEN MATCHED THEN UPDATE SET *
    WHEN NOT MATCHED THEN INSERT *           
    """
    )

    return

#Write silver table 
bronze_stream2silver = (
  spark.readStream.table("default.bronze_pmx_prod")
   .writeStream
   .foreachBatch(process_silver)
   .option("checkpointLocation", checkpoint_silver)
   .trigger(once=True) 
   .start()
)